In [6]:
from google.colab import files
files.upload()


Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"m230296","key":"2365e1865d8bc2db52b4eed2212fcdde"}'}

In [7]:
import os

print("Files before rename:", os.listdir())

# Find and rename kaggle json correctly
for file in os.listdir():
    if file.endswith(".json"):
        os.rename(file, "kaggle.json")

print("Files after rename:", os.listdir())


Files before rename: ['.config', 'kaggle (1) (1).json', 'kaggle .json', 'kaggle (1) (3).json', 'kaggle (1).json', 'kaggle (2).json', 'dataset', 'kaggle (1) (2).json', 'sample_data']
Files after rename: ['.config', 'kaggle.json', 'dataset', 'sample_data']


In [8]:
# Create kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

# Install kaggle
!pip install -q kaggle

# VERIFY FILE CONTENT (IMPORTANT)
!cat /root/.kaggle/kaggle.json


{"username":"m230296","key":"2365e1865d8bc2db52b4eed2212fcdde"}

In [9]:
!kaggle datasets download -d gti-upm/leapgestrecog
!unzip -q leapgestrecog.zip
!ls


Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
 99% 2.10G/2.13G [00:20<00:00, 109MB/s] 
100% 2.13G/2.13G [00:20<00:00, 112MB/s]
dataset  leapgestrecog	leapGestRecog  leapgestrecog.zip  sample_data


In [10]:
import shutil

SOURCE = "leapGestRecog"
DEST = "dataset"

if not os.path.exists(SOURCE):
    raise RuntimeError("❌ Dataset folder not found — Kaggle download failed")

os.makedirs(DEST, exist_ok=True)

for subject in os.listdir(SOURCE):
    subject_path = os.path.join(SOURCE, subject)
    for gesture in os.listdir(subject_path):
        gesture_name = gesture.split('_', 1)[1]
        src = os.path.join(subject_path, gesture)
        dst = os.path.join(DEST, gesture_name)
        os.makedirs(dst, exist_ok=True)
        for img in os.listdir(src):
            shutil.copy(os.path.join(src, img), dst)

print("✅ Dataset reorganized successfully")


✅ Dataset reorganized successfully


In [11]:
!ls dataset


c  down  fist  fist_moved  index  l  ok  palm  palm_moved  thumb


In [13]:
import tensorflow as tf
from tensorflow.keras import layers

IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)


Found 20000 files belonging to 10 classes.
Using 16000 files for training.
Found 20000 files belonging to 10 classes.
Using 4000 files for validation.


In [14]:
print(train_ds.class_names)


['c', 'down', 'fist', 'fist_moved', 'index', 'l', 'ok', 'palm', 'palm_moved', 'thumb']


In [15]:
import tensorflow as tf
from tensorflow.keras import layers

IMG_SIZE = 224
NUM_CLASSES = 10
EPOCHS = 10

# Normalize
normalization = layers.Rescaling(1./255)
train_ds_norm = train_ds.map(lambda x, y: (normalization(x), y)).prefetch(tf.data.AUTOTUNE)
val_ds_norm   = val_ds.map(lambda x, y: (normalization(x), y)).prefetch(tf.data.AUTOTUNE)

# ================= MODEL =================
model = tf.keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ================= TRAIN =================
history = model.fit(
    train_ds_norm,
    validation_data=val_ds_norm,
    epochs=EPOCHS
)

# ================= SAVE MODEL =================
model.save("hand_gesture_cnn.h5")
print("✅ Model saved as hand_gesture_cnn.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,250 (42.61 MB)

 Trainable params: 11,170,250 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 80ms/step - accuracy: 0.8130 - loss: 0.5756 - val_accuracy: 0.9970 - val_loss: 0.0074
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 79ms/step - accuracy: 0.9986 - loss: 0.0052 - val_accuracy: 0.9980 - val_loss: 0.0053
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 40s 80ms/step - accuracy: 0.9973 - loss: 0.0108 - val_accuracy: 0.9992 - val_loss: 0.0016
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - accuracy: 0.9989 - loss: 0.0044 - val_accuracy: 0.9995 - val_loss: 0.0014
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 1.0000 - loss: 3.7852e-05 - val_accuracy: 0.9998 - val_loss: 9.8072e-04
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 78ms/step - accuracy: 1.0000 - loss: 8.6693e-06 - val_accuracy: 0.9998 - val_loss: 0.0010
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 79ms/step - accuracy: 1.0000 - loss: 4.1044e-06 - val_accuracy: 0.9998 - val_loss: 0.0011
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 1.0000 -

✅ Model saved as hand_gesture_cnn.h5


In [18]:
model.save("hand_gesture_cnn.keras")


In [ ]:
from google.colab import files
files.download("hand_gesture_cnn.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>